#Ejemplo Multiplicacion de Matrices en OpenMP

Código para realizar el producto de dos matrices en forma secuencial y paralela usando notación de almacenamiento lineal





In [ ]:
# Código Python, que tiene el código C de la ejecución Axpy.
code = """
/**
 * Producto de matrices secuencial en C usando almacenamiento lineal
 *
 */
#include <iostream>
#include <stdio.h>
#include <stdlib.h>
#include <ctime> 
#include <sys/time.h>
#include <omp.h>



// ----------------------------------------------------------------------------
//------------------------- Macros que miden el tiempo-------------------------

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define T_TOTAL         1
#define T_MULT_SEC      2
#define T_MULT_OMP      3
//-----------------------------------------------------------------------------


// ----------------------------------------------------------------------------
//------------- funciones para realizar el la multiplicación de las matrices---

//#define M 10
//#define N 5
//#define P 10

void mostrarMatriz(int cant_filas, int cant_col, int *matriz)
{
  // Recorrer producto

  for (int i = 0; i < cant_filas; i++) 
  {
    for (int j = 0; j < cant_col; j++) 
    {
      std::cout <<" "<<matriz[i*cant_col+j];
    }
    std::cout <<std::endl;
  }
}
  

void inicializarMatriz(int cant_filas, int cant_col, int *matriz) {
  
  
  int random = 0;
  
  for (int i = 0; i < cant_filas; i++)
  {
    for (int j = 0; j < cant_col; j++)
    {
      random = rand()%5;
      matriz[i*cant_col+j] = random;
      }
  }
}


//--------------Funcion que realiza el la multiplicacion en forma secuencial
void multiplicarMatricesSecuencial(int cant_filasA,int cant_colA,int cant_filasB, int cant_colB, int *matrizA, int *matrizB,int *producto)
{

  // Necesitamos hacer esto para recorrer cada fila  de producto
  for (int i = 0; i < cant_filasA; i++) 
  {
    // Dentro de producto recorremos cada uno de sus columnas
    for (int j = 0; j < cant_colB; j++)
    {
      int suma = 0;
      // recorremos cada una de las columnas de A
      for (int z = 0; z<cant_colA; z++) 
      {
        // Multiplicamos y sumamos resultado
        suma += matrizA[i*cant_colA+z] * matrizB[z*cant_colB+j];
      }
      // Lo acomodamos dentro del producto
      producto[i*cant_colB+j]=suma;
    }
  }  
}

//-------Funcion que realiza el la multiplicacion en forma paralela con OpenMP
void multiplicarMatricesOpenMP(int cant_filasA,int cant_colA,int cant_filasB, int cant_colB, int *matrizA, int *matrizB,int *producto)
{

  //-----------------------------------------------------------------------------------------------
  // Region paralela
  #pragma omp parallel shared(cant_filasA,cant_colA,cant_filasB,cant_colB,matrizA,matrizB,producto)
  {
    
     #pragma omp for schedule(static)
    // Necesitamos hacer esto para recorrer cada fila na de producto
    for (int i = 0; i < cant_filasA; i++) 
    {
    
      // Dentro de producto recorremos cada uno de sus columnas
      for (int j = 0; j < cant_colB; j++) 
      {
        int suma = 0;
        // recorremos cada una de las columnas de A
        for (int z = 0; z<cant_colA; z++) {
        // Multiplicamos y sumamos resultado
        suma += matrizA[i*cant_colA+z] * matrizB[z*cant_colB+j];
      }
      // Lo acomodamos dentro del producto
      producto[i*cant_colB+j]=suma;
      }
    }  
  }
}

int main(int argc, char* argv[]) 
{

  // Leo los parametros.
  if( argc != 5 )
  {
      std::cerr<< " Error en los parametros ."<<argc<<std::endl;
      exit( -1 );
  }

  int M = atoi( argv[1] );
  int N = atoi( argv[2] );
  int P = atoi( argv[3] );
  std::string  mostrarMatrices = argv[4]; 


  // --------------------------------------------

  // Creo las matrices en forma dinamica
  int matrizA[M*N];
  int matrizB[N*P];
  int producto[M*P];
  
  TIEMPO_INI( T_TOTAL );
  
  if (N != N) 
  {
    printf("Columnas de matriz A deben ser igual a filas de matriz B");
    return 0;
  }
  
  srand(time(NULL));
  
  // lleno las matrices con valores aleatorios
  inicializarMatriz(M, N, matrizA);
  inicializarMatriz(N, P, matrizB);

  //-------------- se ejecuta la multiplicacion en forma secuencial--------------
  TIEMPO_INI( T_MULT_SEC );
  multiplicarMatricesSecuencial(M, N, N, P, matrizA,matrizB,producto);
  TIEMPO_FIN( T_MULT_SEC );

  //---------------------------------------------------------------------------

  //--------- se ejecuta la multiplicacion en forma paralela usando OpenMP-----
  TIEMPO_INI( T_MULT_OMP );
  multiplicarMatricesOpenMP(M, N, N, P, matrizA,matrizB,producto);
  TIEMPO_FIN( T_MULT_OMP );

  //---------------------------------------------------------------------------


  TIEMPO_FIN( T_TOTAL );
  
  if (mostrarMatrices=="SI")
  {
    std::cout<<std::endl<<" Matriz A:"<<std::endl;
    mostrarMatriz(M, N, matrizA);

    std::cout<<std::endl<<" Matriz b"<<std::endl;
    mostrarMatriz(N, P, matrizB);

    std::cout<<std::endl<<" Matriz producto:"<<std::endl;
    mostrarMatriz(M, P, producto);
  }

  std::cout<<"----------------------------------------------------------------"<<std::endl;
  std::cout<<"Cantidad de procesadores disponibles: "<<omp_get_num_procs ()<<std::endl;;
  std::cout<<"----------------------------------------------------------------"<<std::endl;
  std::cout<<"Valores Reales  :" <<std::endl;
  std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(T_TOTAL   )<<" [ms]"<<std::endl;
  std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(T_MULT_SEC)<<" [ms]"<<std::endl;
  std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(T_MULT_OMP)<<" [ms]"<<std::endl;
  std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(T_MULT_SEC)<<" / "<<TIEMPO_GET(T_MULT_OMP)<<" = "<<TIEMPO_GET(T_MULT_SEC)/TIEMPO_GET(T_MULT_OMP)<<std::endl;
  std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(T_MULT_SEC)/TIEMPO_GET(T_MULT_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(T_MULT_SEC)/(omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP))<<std::endl;
  std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(T_MULT_SEC)<<" = "<<TIEMPO_GET(T_MULT_SEC)<<std::endl;
  std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(T_MULT_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP)<<std::endl;
  std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP)<<" - "<<TIEMPO_GET(T_MULT_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP))-TIEMPO_GET(T_MULT_SEC)<<std::endl;

  std::cout<<std::endl;
  std::cout<<"----------------------------------------------------------------"<<std::endl;
  std::cout<<"Valores Ideal: "<<std::endl;
  TIEMPO_GET(T_MULT_OMP) = TIEMPO_GET(T_MULT_SEC) / 2;
  std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(T_MULT_SEC)<<" [ms]"<<std::endl;
  std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(T_MULT_OMP)<<" [ms]"<<std::endl;

  std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(T_MULT_SEC)<<" / "<<TIEMPO_GET(T_MULT_OMP)<<" = "<<TIEMPO_GET(T_MULT_SEC)/TIEMPO_GET(T_MULT_OMP)<<std::endl;
  std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(T_MULT_SEC)/TIEMPO_GET(T_MULT_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(T_MULT_SEC)/(omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP))<<std::endl;
  std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(T_MULT_SEC)<<" = "<<TIEMPO_GET(T_MULT_SEC)<<std::endl;
  std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(T_MULT_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP)<<std::endl;
  std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP)<<" - "<<TIEMPO_GET(T_MULT_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(T_MULT_OMP))-TIEMPO_GET(T_MULT_SEC)<<std::endl;

}

"""
text_file = open("mult_matriz_secuencial_estatico.cpp", "w")
text_file.write(code)
text_file.close()

text_file = open("mult_matriz_secuencial_estatico.cpp", "w")
text_file.write(code)
text_file.close()

### 1.2.3. Compilación de código

In [ ]:
!g++ -o mult -fopenmp mult_matriz_secuencial_estatico.cpp 

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Mostrar_Matrices = "NO" #@param ["SI", "NO"]
Num_Thread = 7 #@param {type:"slider", min:0, max:7, step:1}
M = 10 #@param {type:"number"}
N = 10 #@param {type:"number"}
P = 10 #@param {type:"number"}

%env OMP_NUM_THREADS=$Num_Thread
!./mult $M $N $P $Mostrar_Matrices

---
# Medidas de prestaciones en algoritmos paralelos
Las técnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7]. 


## SpeedUp
Referencia a la ganancia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = \frac{Tiempo\_Secuencial}{Tiempo\_Paralelo} $</center>



##Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganancia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = \frac{SpeedUp}{Nro\_Procesadores} $</center>


## Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resolución del problema.

<center>$ coste = Nro\_Procesadores \times Tiempo\_Algoritmo$</center>


## Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

<center>$ Overhead = \frac{Coste}{Tiempo\_Secuencial} $</center>

---

In [ ]:
!cat /proc/cpuinfo

---
# Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[4] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[5] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://so-unlam.com.ar/material-clase/HPC/Arquitecturas_de_computadores_avanzadas_(Modulo_1).pdf)